# ASSIGNMENT:

# Setup/Test Environment:

In [ ]:
from web_footprinting_utils import setup_ipython
import os

get_ipython, INTERACTIVE = setup_ipython()

In [ ]:
import sys
sys.executable

'/root/miniconda/bin/python'

In [1]:
%env TARGET=10.11.1.95

env: TARGET=10.11.1.95


In [ ]:
!echo Make sure we are connected to VPN:
result = !ip a | grep tun0

if not len(result)>0:
    print("Not connected!")
#     input()
else:
    print("Connected!")
    print(result)

Make sure we are connected to VPN:
Not connected!


# Initial (Unreliable) RustScan:

In [5]:
import os
pwd = os.getcwd()
rustscan = "docker run -v '%s':/workspace  --volume /etc/passwd:/etc/passwd:ro --volume /etc/group:/etc/group:ro --user $(id -u) --rm my-other-github-account/rustscan" % pwd

In [6]:
!{rustscan} -t 4000 -r 1-65535 -a {os.environ["TARGET"]} -- -Pn -A -sV

.----. .-. .-. .----..---.  .----. .---.   .--.  .-. .-.
| {}  }| { } |{ {__ {_   _}{ {__  /  ___} / {} \ |  `| |
| .-. \| {_} |.-._} } | |  .-._} }\     }/  /\  \| |\  |
`-' `-'`-----'`----'  `-'  `----'  `---' `-'  `-'`-' `-'
The Modern Day Port Scanner.
________________________________________
: https://discord.gg/GFrQsGy           :
: https://github.com/RustScan/RustScan :
 --------------------------------------
Nmap? More like slowmap.🐢

[~] The config file is expected to be at "/root/.rustscan.toml"
[~] File limit higher than batch size. Can increase speed by increasing batch size '-b 1048476'.
Open 10.10.76.239:22
Open 10.10.76.239:53
Open 10.10.76.239:8009
Open 10.10.76.239:8080
[~] Starting Script(s)
[>] Running script "nmap -vvv -p {{port}} {{ip}} -Pn -A -sV" on ip 10.10.76.239
Depending on the complexity of the script, results may take some time to appear.
[~] Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-14 04:10 UTC
NSE: Loaded 151 scripts for scanning.
NSE: Script Pr

# Initial Nmap:

In [2]:
!echo $TARGET
!mkdir -p scans/quick scans/tcp scans/udp scans/scripts scans/http scans/no_ping

10.11.1.95


In [8]:
!ping -c 5 $TARGET

PING 10.10.76.239 (10.10.76.239) 56(84) bytes of data.
64 bytes from 10.10.76.239: icmp_seq=1 ttl=64 time=0.460 ms
64 bytes from 10.10.76.239: icmp_seq=2 ttl=64 time=0.465 ms
64 bytes from 10.10.76.239: icmp_seq=3 ttl=64 time=0.432 ms
64 bytes from 10.10.76.239: icmp_seq=4 ttl=64 time=0.455 ms
64 bytes from 10.10.76.239: icmp_seq=5 ttl=64 time=0.442 ms

--- 10.10.76.239 ping statistics ---
5 packets transmitted, 5 received, 0% packet loss, time 4057ms
rtt min/avg/max/mdev = 0.432/0.450/0.465/0.012 ms


# Quick Scan:

In [3]:
!sudo nmap -vv -oA scans/quick/quick_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-18 15:57 PDT
Initiating Ping Scan at 15:57
Scanning 10.11.1.95 [4 ports]
Completed Ping Scan at 15:57, 0.14s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 15:57
Completed Parallel DNS resolution of 1 host. at 15:57, 0.01s elapsed
Initiating SYN Stealth Scan at 15:57
Scanning 10.11.1.95 [1000 ports]
Discovered open port 25/tcp on 10.11.1.95
Discovered open port 22/tcp on 10.11.1.95
Discovered open port 110/tcp on 10.11.1.95
Discovered open port 119/tcp on 10.11.1.95
Completed SYN Stealth Scan at 15:57, 1.82s elapsed (1000 total ports)
Nmap scan report for 10.11.1.95
Host is up, received reset ttl 63 (0.12s latency).
Scanned at 2021-10-18 15:57:35 PDT for 2s
Not shown: 996 closed ports
Reason: 996 resets
PORT    STATE SERVICE REASON
22/tcp  open  ssh     syn-ack ttl 63
25/tcp  open  smtp    syn-ack ttl 63
110/tcp open  pop3    syn-ack ttl 63
119/tcp open  nntp    syn-ack ttl 63

Read data files from: /usr/bin/.

# All Ports:

In [4]:
!sudo nmap -p- -sV -O -vv -oA  scans/tcp/tcp_scan $TARGET

Starting Nmap 7.91 ( https://nmap.org ) at 2021-10-18 15:57 PDT
NSE: Loaded 45 scripts for scanning.
Initiating Ping Scan at 15:57
Scanning 10.11.1.95 [4 ports]
Completed Ping Scan at 15:58, 0.12s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 15:58
Completed Parallel DNS resolution of 1 host. at 15:58, 0.02s elapsed
Initiating SYN Stealth Scan at 15:58
Scanning 10.11.1.95 [65535 ports]
Discovered open port 22/tcp on 10.11.1.95
Discovered open port 110/tcp on 10.11.1.95
Discovered open port 25/tcp on 10.11.1.95
SYN Stealth Scan Timing: About 16.68% done; ETC: 16:01 (0:02:35 remaining)
Discovered open port 4555/tcp on 10.11.1.95
SYN Stealth Scan Timing: About 35.80% done; ETC: 16:01 (0:02:16 remaining)
SYN Stealth Scan Timing: About 47.91% done; ETC: 16:01 (0:01:55 remaining)
SYN Stealth Scan Timing: About 64.50% done; ETC: 16:01 (0:01:15 remaining)
SYN Stealth Scan Timing: About 75.43% done; ETC: 16:01 (0:00:54 remaining)
Discovered open port 119/tcp on 10.11.

In [11]:
import xmltodict

with open('scans/tcp/tcp_scan.xml','r') as f:
    xml_dict = xmltodict.parse(f.read())

In [12]:
PING=True
try:
    port_list = [port['@portid'] for port in xml_dict['nmaprun']['host']['ports']['port']]
    %env PORTS={",".join(port_list)}
except:
    print("Host is down")
    PING=False

env: PORTS=22,53,8009,8080


# No Ping:

In [13]:
!sudo nmap -sV -Pn -vv -oA scans/no_ping/no_ping_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-14 04:11 UTC
NSE: Loaded 45 scripts for scanning.
Initiating ARP Ping Scan at 04:11
Scanning 10.10.76.239 [1 port]
Completed ARP Ping Scan at 04:11, 0.03s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 04:11
Completed Parallel DNS resolution of 1 host. at 04:11, 0.00s elapsed
Initiating SYN Stealth Scan at 04:11
Scanning ip-10-10-76-239.eu-west-1.compute.internal (10.10.76.239) [1000 ports]
Discovered open port 22/tcp on 10.10.76.239
Discovered open port 8080/tcp on 10.10.76.239
Discovered open port 53/tcp on 10.10.76.239
Discovered open port 8009/tcp on 10.10.76.239
Completed SYN Stealth Scan at 04:11, 0.10s elapsed (1000 total ports)
Initiating Service scan at 04:11
Scanning 4 services on ip-10-10-76-239.eu-west-1.compute.internal (10.10.76.239)
Completed Service scan at 04:11, 6.02s elapsed (4 services on 1 host)
NSE: Script scanning 10.10.76.239.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 04:1

In [14]:
if not PING:
    !sudo nmap -sV -Pn -p- -vv -oA  scans/tcp/tcp_scan $TARGET

# Run Default Scripts:

In [15]:
if PING:
    !sudo nmap -sC -p $PORTS -vv -oA scans/scripts/script_scan $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-14 04:11 UTC
NSE: Loaded 121 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 2) scan.
Initiating NSE at 04:11
Completed NSE at 04:11, 0.00s elapsed
NSE: Starting runlevel 2 (of 2) scan.
Initiating NSE at 04:11
Completed NSE at 04:11, 0.00s elapsed
Initiating ARP Ping Scan at 04:11
Scanning 10.10.76.239 [1 port]
Completed ARP Ping Scan at 04:11, 0.04s elapsed (1 total hosts)
Initiating Parallel DNS resolution of 1 host. at 04:11
Completed Parallel DNS resolution of 1 host. at 04:11, 0.00s elapsed
Initiating SYN Stealth Scan at 04:11
Scanning ip-10-10-76-239.eu-west-1.compute.internal (10.10.76.239) [4 ports]
Discovered open port 8080/tcp on 10.10.76.239
Discovered open port 22/tcp on 10.10.76.239
Discovered open port 53/tcp on 10.10.76.239
Discovered open port 8009/tcp on 10.10.76.239
Completed SYN Stealth Scan at 04:11, 0.03s elapsed (4 total ports)
NSE: Script scanning 10.10.76.239.
NSE: Starting runlevel 

In [16]:
if PING:
    !sudo nmap -A -T5 -vv --script=default,safe,discovery $TARGET
else:
    !sudo nmap -A -T5 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-14 04:11 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 04:11
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
too short
too short
NSE Timing: About 96.00% done; ETC: 04:11 (0:00:01 remaining)
Completed NSE at 04:11, 40.10s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 04:11
Completed NSE at 04:11, 0.00s elapsed
NSE: Starting runlevel 3 (

In [17]:
if PING:
    !sudo nmap -A -T3 -vv --script=default,safe,discovery $TARGET
else:
    !sudo nmap -A -T3 -Pn -vv --script=default,safe,discovery $TARGET

Starting Nmap 7.80 ( https://nmap.org ) at 2021-10-14 04:12 UTC
NSE: Loaded 417 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 3) scan.
Initiating NSE at 04:12
NSE: [url-snarf] no network interface was supplied, aborting ...
NSE: [targets-xml] Need to supply a file name with the targets-xml.iX argument
NSE: [broadcast-sonicwall-discover] No network interface was supplied, aborting.
NSE: [shodan-api] Error: Please specify your ShodanAPI key with the shodan-api.apikey argument
NSE: [targets-ipv6-map4to6] This script is IPv6 only.
NSE: [targets-ipv6-wordlist] Need to be executed for IPv6.
NSE: [mtrace] A source IP must be provided through fromip argument.
NSE: [broadcast-ataoe-discover] No interface supplied, use -e
too short
too short
NSE Timing: About 96.00% done; ETC: 04:13 (0:00:01 remaining)
Completed NSE at 04:13, 40.09s elapsed
NSE: Starting runlevel 2 (of 3) scan.
Initiating NSE at 04:13
Completed NSE at 04:13, 0.00s elapsed
NSE: Starting runlevel 3 (

In [ ]:
from getpass import getpass
import os

if PING:
    !sudo nmap -A -p- -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}
else:
    !sudo nmap -A -p- -Pn -T5 -vv --script=default,safe,discovery {os.environ['TARGET']}

In [ ]:
UDP Scan, this takes forever...

from getpass import getpass

!sudo nmap -sU --open -oA scans/udp/udp_scan $TARGET
!sudo nmap -sU --open -p- -oA scans/udp/udp_scan $TARGET